In [1]:
# Importar ferramentas
import pandas as pd
import numpy as np

# Ler arquivo .csv extraído da Alesp
df = pd.read_csv('despesas.csv', encoding='latin-1', low_memory=False)

# Filtrar dados para a categoria e o ano desejado
df_2017 = df[df['Ano'] == 2017]
data = df_2017[df_2017['Tipo'] == str('D - MATERIAIS E SERVIÇOS GRÁFICOS, DE CÓPIAS  E REPRODUÇÃO DE DOCS')]

# Checar se está tudo bem
data.head(10)

,Deputado,Matricula,Ano,Mes,Tipo,CNPJ,Fornecedor,Valor
140,ABELARDO CAMARINHA,300257,2017,1,"D - MATERIAIS E SERVIÇOS GRÁFICOS, DE CÓPIAS ...",03136828000160,MARCO ANTONIO D AVILA,8000.0
145,ABELARDO CAMARINHA,300257,2017,2,"D - MATERIAIS E SERVIÇOS GRÁFICOS, DE CÓPIAS ...",03136828000160,MARCO ANTONIO D AVILA,8000.0
151,ABELARDO CAMARINHA,300257,2017,3,"D - MATERIAIS E SERVIÇOS GRÁFICOS, DE CÓPIAS ...",03136828000160,MARCO ANTONIO D AVILA,8000.0
160,ABELARDO CAMARINHA,300257,2017,4,"D - MATERIAIS E SERVIÇOS GRÁFICOS, DE CÓPIAS ...",03136828000160,MARCO ANTONIO D AVILA,8000.0
168,ABELARDO CAMARINHA,300257,2017,5,"D - MATERIAIS E SERVIÇOS GRÁFICOS, DE CÓPIAS ...",03136828000160,MARCO ANTONIO D AVILA,8000.0
175,ABELARDO CAMARINHA,300257,2017,6,"D - MATERIAIS E SERVIÇOS GRÁFICOS, DE CÓPIAS ...",03136828000160,MARCO ANTONIO D AVILA,8000.0
182,ABELARDO CAMARINHA,300257,2017,7,"D - MATERIAIS E SERVIÇOS GRÁFICOS, DE CÓPIAS ...",03136828000160,MARCO ANTONIO D AVILA,8000.0
190,ABELARDO CAMARINHA,300257,2017,8,"D - MATERIAIS E SERVIÇOS GRÁFICOS, DE CÓPIAS ...",03136828000160,MARCO ANTONIO D AVILA,8000.0
196,ABELARDO CAMARINHA,300257,2017,9,"D - MATERIAIS E SERVIÇOS GRÁFICOS, DE CÓPIAS ...",03136828000160,MARCO ANTONIO D AVILA,8000.0
203,ABELARDO CAMARINHA,300257,2017,10,"D - MATERIAIS E SERVIÇOS GRÁFICOS, DE CÓPIAS ...",03136828000160,MARCO ANTONIO D AVILA,8000.0


In [2]:
# Mostrar informações
print('Quantidade de notas fiscais em análise:', len(data))
print('Quantidade de deputados em análise:', data['Deputado'].nunique())
print('Valor em análise: R$', data['Valor'].sum())

# Mostrar estatísticas gerais
print('Valor médio da nota fiscal: R$', data['Valor'].mean())
print('Maior valor de nota fiscal: R$', data['Valor'].max())
print('Menor valor da nota fiscal: R$', data['Valor'].min())
print('25% das notas têm valores inferiores a R$', data['Valor'].quantile(.25))
print('50% das notas têm valores inferiores a R$', data['Valor'].quantile(.50))
print('75% das notas têm valores inferiores a R$', data['Valor'].quantile(.75))

Quantidade de notas fiscais em análise: 817
Quantidade de deputados em análise: 91
Valor em análise: R$ 4372527.649999999
Valor médio da nota fiscal: R$ 5351.931028151774
Maior valor de nota fiscal: R$ 8000.0
Menor valor da nota fiscal: R$ 5.7
25% das notas têm valores inferiores a R$ 2470.0
50% das notas têm valores inferiores a R$ 7030.26
75% das notas têm valores inferiores a R$ 7900.0


In [3]:
# Tomar apenas as notas com maiores valores (ou seja, acima de R$ 7.030,26)
above_50_percentil = data[data['Valor'] >= data['Valor'].quantile(.50)]

In [4]:
# Ver quantas vezes em 2017 cada deputado desembolsou acima de R$ 7.030,26
def find_sum_of_values(above_50_percentil, aggregator, property):
   
    total_label = '{}_total'.format(property)
    max_label = '{}_max'.format(property)
    mean_label = '{}_mean'.format(property)
    min_label = '{}_min'.format(property)

    result = {
        'ocorrencias': [],
        aggregator: above_50_percentil[aggregator].unique(),
        max_label: [],
        mean_label: [],
        min_label: [],
        total_label: []       
    }
    
    for item in result[aggregator]:
        if isinstance(above_50_percentil[aggregator].iloc[0], str):
            item = str(item)
        values = above_50_percentil[above_50_percentil[aggregator] == item]
        property_total = int(values[property].sum())
        ocorrencias = int(values[property].count())
        
        result[total_label].append(property_total)
        result['ocorrencias'].append(ocorrencias)
        result[mean_label].append(property_total/ocorrencias)
        result[max_label].append(np.max(values[property]))
        result[min_label].append(np.min(values[property]))

    return pd.DataFrame(result).sort_values(by=aggregator)

df = find_sum_of_values(above_50_percentil, "Deputado", "Valor")

# Selecionar apenas deputados que gastaram o valor de corte por 9 meses ou mais
final_result = df[df['ocorrencias'] >= 9]
final_result

,Deputado,Valor_max,Valor_mean,Valor_min,Valor_total,ocorrencias
0,ABELARDO CAMARINHA,8000.00,8000.000000,8000.0,96000,12
4,ANA DO CARMO,8000.00,7950.000000,7800.0,95400,12
7,BETH SAHÃO,8000.00,8000.000000,8000.0,96000,12
8,CAIO FRANÇA,7840.00,7718.888889,7120.0,69470,9
12,CAUÊ MACRIS,7980.00,7980.000000,7980.0,79800,10
15,CEZINHA DE MADUREIRA,7980.00,7967.272727,7960.0,87640,11
24,ENIO TATTO,8000.00,7460.000000,7200.0,74600,10
27,FERNANDO CURY,8000.00,8000.000000,8000.0,96000,12
34,JORGE WILSON XERIFE DO CONSUMIDOR,7988.00,7857.200000,7740.0,78572,10
36,JOSÉ ZICO PRADO,8000.00,7995.636364,7952.0,87952,11


In [5]:
# Estes são os deputados que iremos investigar. :)

# Baixar para .csv
final_result.to_csv('resultado_apurado.csv', sep=';', encoding='utf-8')